# 낭찬하우스 가계부 작성
- banksalad app을 통해 다양한 금융기관의 지출, 수입 데이터를 통합 연동 및 수집한다.
- 수집한 지출 및 수입 데이터를 banksalad app을 통해 메일로 data를 export한다.
- mail로 보내진 data를 download 및 import 하여 자동처리를 진행한다.
- 자동처리 되지 않은 data를 수동으로 입력한다. (+카카오페이 지출 데이터)
- 처리된 데이터(xls파일을) whooing 가계부 web에 import 시킨다.


In [ ]:
import pandas as pd
import os
import numpy as np

### 뱅크샐러드 Raw Data Import

In [ ]:
# 메일로 다운받은 파일 import 하기
fPath = input()
fName = input()
df = pd.read_excel('{}\\{}.xlsx'.format(fPath, fName), 
                   sheet_name='가계부 내역'
                  )

In [ ]:
# banksalad의 data export 는 자동으로 최근 1년이 설정됨
# whooing 가계부에 데이터가 기록된 가장 마지막 날짜 이후 데이터만을 스크립트 대상으로 삼는다.
lastDate_in_Whooing = input('마지막 후잉가계부 업데이트일 (yyyymmdd): : ')

df2 = df[df['날짜']>=lastDate_in_Whooing]

### DataSet 확인

In [ ]:
print('\nDatasize:',df2.shape)
print('\nDatatype \n',df2['타입'].value_counts())

print('\nCurrency:',df2['화폐'].value_counts()) #KRW 이외의 화폐가 존재한다면 별도 처리 필요

print('\nPayment method\n',df2['결제수단'].value_counts())

### Column 정리

In [ ]:
df2.drop('메모', axis=1, inplace=True)

df2.rename({'내용':'메모'}, axis=1, inplace = True)

df2 = df2.reindex(['날짜',
                        '시간',
                        #'대분류',
                        #'소분류',                             
                        '아이템(괄호)',                             
                        '금액',
                        #'화폐',                                                         
                        '왼쪽',
                        '오른쪽',
                        '메모',            
                        '결제수단',
                        '타입',
                        ], axis=1)

### Data 정리


In [ ]:
df2.drop(df2.loc[(df2['금액']>0)&(df2['타입']=='지출')].index, inplace=True) #신용카드 취소로 추정되는 데이터 제외

df2.drop(df2.loc[(df2['금액']==0)].index, inplace=True) #금액이 0원인 내역들 제외

- 각 결제수단별 Debit / Credit 분류
- Debit / Credit 분류 이후 금액 모두 양수 처리

In [ ]:
###
df2.loc[(df2['금액']<0)&(df2['결제수단']=='찬우 신용카드'),'오른쪽'] = '신용카드(우리/508867)'
df2.loc[(df2['금액']<0)&(df2['결제수단']=='찬우 신용카드'),'금액'] = df2.loc[(df2['금액']<0)&(df2['결제수단']=='찬우 신용카드'),'금액']*-1


###
df2.loc[(df2['금액']>0)&(df2['결제수단'].isin(['우리V체크카드','우리꿈통장','위비 Five 체크','찬우 생활비', '나은 생활비'])),'왼쪽'] = '생활비통장(우리/875309)'
df2.loc[(df2['금액']<0)&(df2['결제수단'].isin(['우리V체크카드','우리꿈통장','위비 Five 체크','찬우 생활비', '나은 생활비'])),'오른쪽'] = '생활비통장(우리/875309)'
df2.loc[(df2['금액']<0)&(df2['결제수단'].isin(['우리V체크카드','우리꿈통장','위비 Five 체크','찬우 생활비', '나은 생활비'])),'금액'] = df2.loc[(df2['금액']<0)&(df2['결제수단'].isin(['우리V체크카드','우리꿈통장','위비 Five 체크','찬우 생활비', '나은 생활비'])),'금액']*-1


###
df2.loc[(df2['금액']>0)&(df2['결제수단'].isin(['우리신세대플러스통장'])),'왼쪽'] = '마이너스통장(우리/508867)'
df2.loc[(df2['금액']<0)&(df2['결제수단'].isin(['우리신세대플러스통장'])),'오른쪽'] = '마이너스통장(우리/508867)'
df2.loc[(df2['금액']<0)&(df2['결제수단'].isin(['우리신세대플러스통장'])),'금액'] = df2.loc[(df2['금액']<0)&(df2['결제수단'].isin(['우리신세대플러스통장'])),'금액']*-1


###
df2.loc[(df2['금액']>0)&(df2['결제수단'].isin(['우리SUPER주거래통장'])),'왼쪽'] = '급여통장(우리/688431)'
df2.loc[(df2['금액']<0)&(df2['결제수단'].isin(['우리SUPER주거래통장'])),'오른쪽'] = '급여통장(우리/688431)'
df2.loc[(df2['금액']<0)&(df2['결제수단'].isin(['우리SUPER주거래통장'])),'금액'] = df2.loc[(df2['금액']<0)&(df2['결제수단'].isin(['우리SUPER주거래통장'])),'금액']*-1


###
df2.loc[(df2['금액']>0)&(df2['결제수단'].isin(['주거래 하나 통장'])),'왼쪽'] = '하나은행_예금계좌'
df2.loc[(df2['금액']<0)&(df2['결제수단'].isin(['주거래 하나 통장'])),'오른쪽'] = '하나은행_예금계좌'
df2.loc[(df2['금액']<0)&(df2['결제수단'].isin(['주거래 하나 통장'])),'금액'] = df2.loc[(df2['금액']<0)&(df2['결제수단'].isin(['주거래 하나 통장'])),'금액']*-1


- Debit/Credit 처리가 빠진 거래가 있는지 확인

In [ ]:
df2.loc[(df2['왼쪽'].isnull()) & (df2['오른쪽'].isnull())]

### 자주사용하는 거래 일괄 처리

수입 관련 거래

In [ ]:
df2.loc[(df2['오른쪽'].isnull()) & (df2['메모'].str.contains('급여')), ['오른쪽','아이템(괄호)']] = ['월급','급여']
df2.loc[(df2['오른쪽'].isnull()) & (df2['메모'].str.contains('나은')), ['오른쪽','아이템(괄호)']] = ['from나은','나은']
df2.loc[(df2['오른쪽'].isnull()) & (df2['메모'].str.contains('예금이자')), ['오른쪽','아이템(괄호)']] = ['이자수익','예금이자']
df2.loc[(df2['오른쪽'].isnull()) & (df2['메모'].str.contains('우리체크캐시백')), ['오른쪽','아이템(괄호)']] = ['기타수익','우리체크캐시백']

In [ ]:
df2.loc[(df2['오른쪽'].isnull())] 

비용 관련 거래

In [ ]:
df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('카페')), ['왼쪽','아이템(괄호)']] = ['식비','커피']
df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('까페')), ['왼쪽','아이템(괄호)']] = ['식비','커피']
df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('커피')), ['왼쪽','아이템(괄호)']] = ['식비','커피']
df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('스타벅스')), ['왼쪽','아이템(괄호)']] = ['식비','커피']
df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('이디야')), ['왼쪽','아이템(괄호)']] = ['식비','커피']

df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('베이커리')), ['왼쪽','아이템(괄호)']] = ['식비','간식(빵)']

df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('우아한형제들')), ['왼쪽','아이템(괄호)']] = ['식비','배달의민족']

df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('신동아마트')), ['왼쪽','아이템(괄호)']] = ['식비','마트']


df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('ＰＣ')), ['왼쪽','아이템(괄호)']] = ['여가','PC방']


df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('택시')), ['왼쪽','아이템(괄호)']] = ['교통비','택시']
df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('고속버스')), ['왼쪽','아이템(괄호)']] = ['교통비','고속버스']


df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('약국')), ['왼쪽','아이템(괄호)']] = ['의료,건강','약국']
df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('병원')), ['왼쪽','아이템(괄호)']] = ['의료,건강','병원']
df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('한의원')), ['왼쪽','아이템(괄호)']] = ['의료,건강','한의원']


df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('KT유선상품 자동')), ['왼쪽','아이템(괄호)']] = ['통신비','KT인터넷+TV']
df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('KT통신요금 자동')), ['왼쪽','아이템(괄호)']] = ['통신비','핸드폰']


df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('우리카드결제')), ['왼쪽','아이템(괄호)']] = ['신용카드(508867)','신용카드']

df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('카카오페이')), ['왼쪽','아이템(괄호)']] = ['카카오페이','이체']
df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('벗얼라이브')), ['왼쪽','아이템(괄호)']] = ['식비','커피']
df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('ＧＳ２５')), ['왼쪽','아이템(괄호)']] = ['생활용품','편의점']
df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('지에스리테일')), ['왼쪽','아이템(괄호)']] = ['식비','마트']


df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('쏘카')), ['왼쪽','아이템(괄호)']] = ['교통비','쏘카']
df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('파리바게뜨')), ['왼쪽','아이템(괄호)']] = ['식비','간식(빵)']
df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('파리크라상')), ['왼쪽','아이템(괄호)']] = ['식비','간식(빵)']
df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('피부과')), ['왼쪽','아이템(괄호)']] = ['의료,건강','피부과']
df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('버킷플레이스')), ['왼쪽','아이템(괄호)']] = ['생활용품','오늘의집']
df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('노브랜드')), ['왼쪽','아이템(괄호)']] = ['식비','마트']

df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('GS25')), ['왼쪽','아이템(괄호)']] = ['생활용품','편의점']
df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('지에스25')), ['왼쪽','아이템(괄호)']] = ['생활용품','편의점']

df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('씨제이올리브네트')), ['왼쪽','아이템(괄호)']] = ['생활용품','올리브영']

df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('(주)신세계페이먼츠')), ['왼쪽','아이템(괄호)']] = ['식비','SSG닷컴']


df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('준오헤어')), ['왼쪽','아이템(괄호)']] = ['의복,미용','머리']

df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('교통-버스')), ['왼쪽','아이템(괄호)']] = ['교통비','버스']
df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('교통-지하철')), ['왼쪽','아이템(괄호)']] = ['교통비','지하철']

df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('아성다이소')), ['왼쪽','아이템(괄호)']] = ['생활용품','다이소']

df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('WORKFLOWY')), ['왼쪽','아이템(괄호)']] = ['지식,문화','Workflowy']
df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('WORKFLOWY')), ['왼쪽','아이템(괄호)']] = ['지식,문화','Workflowy']


df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('신한20')), ['왼쪽','아이템(괄호)']] = ['보험','보험']
df2.loc[(df2['왼쪽'].isnull()) & (df2['메모'].str.contains('DB손')), ['왼쪽','아이템(괄호)']] = ['보험','보험']


In [ ]:
df2.loc[(df2['왼쪽'].isnull())] 

### 정리 결과 Export

In [ ]:
df2.to_excel('{}\\whooing_import.xls'.format(fPath), index=False)